In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

from unsloth import FastLanguageModel
from peft import PeftModel
from huggingface_hub import login, HfApi, create_repo

In [ ]:
TOKEN = None
ORG_PATH = "StefanCoder1/Granite-tuned" 
NEW_PATH = "StefanCoder1/Qwen-tunded"

def load_new(path = ORG_PATH):
    max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = False

    org_model, org_tokenizer = FastLanguageModel.from_pretrained(
          model_name = "unsloth/Qwen2-0.5B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
          # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
      )

    model = PeftModel.from_pretrained(
    org_model,
    path, 
    token=TOKEN,
    )
    merged = model.merge_and_unload()

    return merged, org_tokenizer

def create_HF_repo(repo_name, token=TOKEN):
    login(token=TOKEN) # dont steal it please :((((
    api = HfApi()
    user = api.whoami()["name"]
    repo_id = f"{repo_name}"
    # Ensure repo exists
    create_repo(repo_id, private=True, exist_ok=True)
    return repo_id

repo_id =create_HF_repo(NEW_PATH)
model, tokenizer = load_new(ORG_PATH)

==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
#model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
model.push_to_hub(NEW_PATH, tokenizer, token =TOKEN)
tokenizer.push_to_hub(NEW_PATH,use_auth_token=TOKEN, 
)
